In [ ]:
# Import packages
import os
from matplotlib import pyplot as plt
import pandas as pd
import datetime
import matplotlib as mpl


# Import AuTuMN modules
from autumn.settings import Models, Region
from autumn.settings.folders import OUTPUT_DATA_PATH
from autumn.tools.project import get_project
from autumn.tools import db
from autumn.tools.plots.calibration.plots import calculate_r_hats, get_output_from_run_id
from autumn.tools.plots.uncertainty.plots import _plot_uncertainty, _get_target_values
from autumn.tools.plots.plotter.base_plotter import COLOR_THEME
from autumn.tools.plots.utils import get_plot_text_dict, change_xaxis_to_date, REF_DATE, ALPHAS, COLORS, _apply_transparency, _plot_targets_to_axis

from autumn.dashboards.calibration_results.plots import get_uncertainty_df

In [ ]:
# Specify model details
model = Models.COVID_19
region = Region.SRI_LANKA
dirname = "2022-08-02"

In [ ]:
# get the relevant project and output data
project = get_project(model, region)
project_calib_dir = os.path.join(
    OUTPUT_DATA_PATH, "calibrate", project.model_name, project.region_name
)
calib_path = os.path.join(project_calib_dir, dirname)
# Load tables
mcmc_tables = db.load.load_mcmc_tables(calib_path)
mcmc_params = db.load.load_mcmc_params_tables(calib_path)

uncertainty_df = get_uncertainty_df(calib_path, mcmc_tables, project.plots)
scenario_list = uncertainty_df['scenario'].unique()

# make output directories
output_dir = f"{model}_{region}_{dirname}"
base_dir = os.path.join("outputs", output_dir)
os.makedirs(base_dir, exist_ok=True)
dirs_to_make = ["calibration", "MLE", "median", "uncertainty", "csv_files"]
for dir_to_make in dirs_to_make:
    os.makedirs(os.path.join(base_dir, dir_to_make), exist_ok=True)

In [ ]:
output_name = "hospital_admissions"
scenario_idx = 0
mask = (
(uncertainty_df["type"] == output_name)
& (uncertainty_df["scenario"] == scenario_idx)
)
df = uncertainty_df[mask]
times = df.time.unique()[1:]
quantiles = {}
quantile_vals = df["quantile"].unique().tolist()
for q in quantile_vals:
    mask = df["quantile"] == q
    quantiles[q] = df[mask]["value"].tolist()[1:]

hospital_admission_vals = quantiles[0.5]  #median hospital admissions



In [ ]:
##get weekly hospital admissions per 100000
pop_size = 22.156 * 1e6 # SL populaiton size 
per_100000_admissions = [x / pop_size for x in hospital_admission_vals]
per_100000_admissions = [x * 1e5 for x in [x / pop_size for x in hospital_admission_vals]]

weekly_admissions = [x * 7 for x in per_100000_admissions]

In [ ]:
fig = plt.figure(figsize=(18, 7))
plt.style.use("ggplot")
axis = fig.add_subplot(1,2,1)
label_font_size = 22

plt.plot(times,weekly_admissions,color = 'k')
axis.set_xticks([495, 507, 599])
axis.set_xlim((380, 700))
change_xaxis_to_date(axis, REF_DATE)
plt.xticks(rotation = 90)
# WHO threshold
plt.axhline(y = 31, color = 'r', linestyle = '--')
plt.text(385, 33, r'WHO threshold', fontsize = 18)

# plot the times in which actual lockdowns were implemented in SL
# second lockdown implemented on 21st May 2021 -> 508
# thrid lockdown implemented - 21st August -> 599
times_array = times.tolist()
time_index_second_lockdown= times_array.index(50)
admissions_second_lockdown = weekly_admissions[time_index_second_lockdown]
print(admissions_second_lockdown)



# plot the time in which WHO threshold is exceeded
plt.vlines(x=[495], ymin=[0], ymax=[31], colors='grey', ls='--', lw=2.5, ) # the timings in which the WHO threshold is exceeded
plt.vlines(x=[508, 599], ymin=[0], ymax=[54, 171], colors='purple', ls='--', lw=2.5, ) # the timings in which the actual lockdowns were implemented

plt.hlines(y=[54, 171], xmin=[200], xmax=[508, 599], colors='purple', ls='--', lw=2.5, )

plt.setp(axis.get_yticklabels(), fontsize=label_font_size)
plt.setp(axis.get_xticklabels(), fontsize=label_font_size)
plt.ylabel('Daily COVID-19 hospital admissions \n per 100,000 per week', fontsize=label_font_size)
plt.xlabel('Date', fontsize=label_font_size)